#### 스타벅스 서울시 매장위치 크롤링

##### 스타벅스 웹사이트
- https://www.starbucks.co.kr/index.do 

##### 셀레니움 등 라이브러리 임포트

In [48]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
from tqdm.autonotebook import tqdm

In [2]:
# 크롬 웹드라이버 실행
driver = webdriver.Chrome()
url = 'https://www.starbucks.co.kr/store/store_map.do?disp=locale'
driver.get(url)

In [5]:
# 웹드라이버 선택할 위치값 작성(20년 8월과 동일)
btn_seoul = '#container > div > form > fieldset > div > section > article.find_store_cont > ' + \
            'article > article:nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > ' + \
            'li:nth-child(1) > a'
driver.find_element(By.CSS_SELECTOR, btn_seoul).click()

In [7]:
# 서울이 선택된 상태
# '전체' 클릭
btn_all = '#mCSB_2_container > ul > li > a' # 첫번째 값이기 때문에 nth-child 도 필요없음
driver.find_element(By.CSS_SELECTOR, btn_all).click()

In [9]:
# 스타벅스는 한화면에 전부 리스트업되기때문에 함수를 만들필요도 없음
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
soup

In [11]:
# 검색결과 위치 
starbuks_soup_list = soup.select('li.quickResultLstCon')
len(starbuks_soup_list)

600

In [12]:
starbuks_soup_list[0]

<li class="quickResultLstCon" data-code="3762" data-hlytag="null" data-index="0" data-lat="37.501087" data-long="127.043069" data-name="역삼아레나빌딩" data-storecd="1509" style="background:#fff"> <strong data-my_siren_order_store_yn="N" data-name="역삼아레나빌딩" data-store="1509" data-yn="N">역삼아레나빌딩  </strong> <p class="result_details">서울특별시 강남구 언주로 425 (역삼동)<br/>1522-3232</p> <i class="pin_general">리저브 매장 2번</i></li>

In [47]:
# 매장정보 확인
starbuks_store = starbuks_soup_list[0]
name = starbuks_store.select('strong')[0].text.strip()
lat = starbuks_store['data-lat'].strip()
lng = starbuks_store['data-long'].strip()
store_type = starbuks_store.select('i')[0]['class'][0].split('_')[1] # _로 자르면 두번째값이 general/reserve
address = starbuks_store.select('p')[0].get_text(separator='|').split('|')[0]

print(name) # 매장명
print(lat, lng, sep=' / ') # 매장 지도 위치
print(store_type)
print(address)

역삼아레나빌딩
37.501087 / 127.043069
general
서울특별시 강남구 언주로 425 (역삼동)


In [49]:
# 600개 매장 데이터 가져오기
starbuks_seoul_list = []

for item in tqdm(starbuks_soup_list):
    name = item.select('strong')[0].text.strip()
    lat = item['data-lat'].strip()
    lng = item['data-long'].strip()
    store_type = item.select('i')[0]['class'][0].split('_')[1] # _로 자르면 두번째값이 general/reserve
    address = item.select('p')[0].get_text(separator='|').split('|')[0]

    starbuks_seoul_list.append([name, lat, lng, store_type, address])

100%|██████████| 600/600 [00:00<00:00, 7610.46it/s]


In [52]:
len(starbuks_seoul_list)

600

In [53]:
# DataFrame 생성
columns = ['매장명', '위도', '경도', '매장타입', '주소']
df = pd.DataFrame(starbuks_seoul_list, columns=columns)
df.tail()

,매장명,위도,경도,매장타입,주소
595,사가정역,37.579594,127.087966,general,서울특별시 중랑구 면목로 310
596,상봉역,37.59689,127.08647,general,서울특별시 중랑구 망우로 307 (상봉동)
597,묵동,37.615368,127.076633,general,"서울특별시 중랑구 동일로 952 (묵동, 로프트원 태릉입구역) 1층"
598,양원역,37.6066536267232,127.106359790053,general,서울특별시 중랑구 양원역로10길 3 (망우동)
599,중화역,37.60170912407773,127.07841136432036,general,서울특별시 중랑구 봉화산로 35


In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   매장명     600 non-null    object
 1   위도      600 non-null    object
 2   경도      600 non-null    object
 3   매장타입    600 non-null    object
 4   주소      600 non-null    object
dtypes: object(5)
memory usage: 23.6+ KB


In [55]:
# 엑셀로 저장
df.to_excel('./seoul_starbuks.xlsx', index=False)